# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
# Write your code below.
%load_ext dotenv
%dotenv



The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [3]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
import os
from glob import glob
import logging


_logs = logging.getLogger(__name__)
if not logging.getLogger().handlers:
	logging.basicConfig(level=logging.INFO)

price_files = glob(os.path.join(os.getenv('PRICE_DATA'), "**/**", "*.parquet"))
_logs.info(f'Found {len(price_files)} price files.')



INFO:__main__:Found 2045 price files.


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [5]:
# Write your code below.
# Load data into Dask dataframe
dd_feat = dd.read_parquet(price_files)

# Normalize column names (replace spaces with underscores)
dd_feat = dd_feat.rename(columns=lambda x: x.replace(' ', '_'))

# Add lag features for Close and Adj_Close
dd_feat['Close_lag_1'] = dd_feat.groupby('ticker')['Close'].shift(1)
dd_feat['Adj_Close_lag_1'] = dd_feat.groupby('ticker')['Adj_Close'].shift(1)

# Add returns based on Close
dd_feat['returns'] = (dd_feat['Close'] / dd_feat['Close_lag_1']) - 1

# Add hi_lo_range (High minus Low)
dd_feat['hi_lo_range'] = dd_feat['High'] - dd_feat['Low']
dd_feat.head()


C:\Users\Admin\AppData\Local\Temp\ipykernel_25160\3543449060.py:9: UserWarning: `meta` is not specified, inferred from partial data.
Please provide `meta` if the result is unexpected.
  Before: .shift(func)
  After:  .shift(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .shift(func, meta=('x', 'f8'))            for series result

  dd_feat['Close_lag_1'] = dd_feat.groupby('ticker')['Close'].shift(1)
C:\Users\Admin\AppData\Local\Temp\ipykernel_25160\3543449060.py:10: UserWarning: `meta` is not specified, inferred from partial data.
Please provide `meta` if the result is unexpected.
  Before: .shift(func)
  After:  .shift(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .shift(func, meta=('x', 'f8'))            for series result

  dd_feat['Adj_Close_lag_1'] = dd_feat.groupby('ticker')['Adj_Close'].shift(1)


,Date,Open,High,Low,Close,Adj_Close,Volume,source,ticker,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
175864,2012-01-19,88.199997,88.889999,87.610001,88.830002,88.830002,894300.0,LH.csv,LH,2012,88.470001,88.470001,0.004069,1.279999
76009,2014-02-26,53.779999,54.049999,53.470001,53.849998,47.856152,2025700.0,ALL.csv,ALL,2014,53.650002,47.678425,0.003728,0.579998
172152,1997-04-21,7.500000,7.812500,7.500000,7.500000,7.500000,5200.0,LH.csv,LH,1997,7.812500,7.812500,-0.040000,0.312500
171503,1994-09-26,31.875000,33.437500,31.250000,33.125000,33.125000,147600.0,LH.csv,LH,1994,32.187500,32.187500,0.029126,2.187500
171166,1993-05-26,45.000000,45.625000,44.687500,45.625000,44.662472,341200.0,LH.csv,LH,1993,45.000000,44.050655,0.013889,0.937500


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [6]:
# Write your code below.

# Convert Dask dataframe to pandas
df = dd_feat.compute()

# Add moving average of returns with 10-day window
df['returns_ma_10'] = df.groupby('ticker')['returns'].transform(
    lambda x: x.rolling(10).mean()
)


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.